# Charts for my website

- Author: Kiril from Mindgraph
- Last meaningful update: 21-11-2025

This notebook creates some `plotly` charts for [my website](www.mindgraph.dk). The charts are displayed as visuals in this notebook and also exported to `.html` files on GitHub so that they can be directly embedded on the website.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import ColumnDataSource, Legend, HoverTool, NumeralTickFormatter
from bokeh.layouts import row

In [2]:
# Importing data [WIP - to be expanded]
ft_afsteminger = pd.read_parquet("output/ft_afstemninger.parquet")

In [3]:
# # Changing to the default color palette in plotly
# pio.templates.default = "plotly"

In [4]:
# Creating a custom color palette with the MG colors for plotly
# Using the "Classy" palette from: https://mycolor.space/?hex=%231EA2B5&sub=1
my_custom_palette = ["#1ea2b5", "#324b4f", "#95b0b5", "#9f8ac3", "#6b588d"]
my_template = pio.templates["plotly_white"].layout.template
my_template.layout.colorway = my_custom_palette
pio.templates["my_custom_template"] = my_template
pio.templates.default = "my_custom_template"

## Figure 1

INSERT CHART WITH % OF VOTES BY FOR/AGAINST/ETC ACROSS TIME - STACKED TO 100%

In [5]:
# Aggregating data for chart
id_cols = ["År", "Stemme"]
cols_keep = id_cols + ["AndelStemmer"]
chart_data = ft_afsteminger[ft_afsteminger["Stemme"] != "Fraværende"].copy()
chart_data["AntalStemmer"] = chart_data.groupby(id_cols)["Sæson"].transform("count")
chart_data["AlleStemmer"] = chart_data.groupby("År")["Sæson"].transform("count")
chart_data["AndelStemmer"] = chart_data["AntalStemmer"] / chart_data["AlleStemmer"]
chart_data = chart_data.drop_duplicates(id_cols)

# Pivoting the data
chart_data = chart_data.pivot_table(
    index="År", columns="Stemme", values="AndelStemmer", aggfunc="sum"
).reset_index()
chart_data = chart_data.fillna(0)

# Reordering the columns
cols_order = ["År", "For", "Imod", "Hverken for eller imod"]
chart_data = chart_data[cols_order]

# Specifying col names in English
col_names = ["Year", "For", "Against", "Neither for nor against"]
chart_data_en = chart_data.copy()
chart_data_en.columns = col_names

In [6]:
def figure_1(data: pd.DataFrame, lang: str):

    # Define cols to use based on language
    if lang == "dk":
        x_col = "År"
        y_axis_label = "Andel af stemmer (%)"
        stacker_name = "Stemme"
        value_name = "Andel af stemmer"
    else:
        x_col = "Year"
        y_axis_label = "Share of votes (%)"
        stacker_name = "Vote"
        value_name = "Share of votes"

    # Build ColumnDataSource
    source = ColumnDataSource(data)

    # Stackers = all columns except Year
    stackers = [c for c in data.columns if c != x_col]
    n_unique = len(data.columns) - 1

    p = figure(
        width=700,
        height=400,
        sizing_mode="stretch_width",
        x_axis_label=x_col,
        y_axis_label=y_axis_label,
    )

    # Create stacked areas
    renderers = p.varea_stack(
        stackers=stackers,
        x=x_col,
        color=my_custom_palette[:n_unique],
        legend_label=stackers,
        source=source,
        alpha=0.6,
    )

    # Add hover tooltips for stackers
    hover = HoverTool(
        renderers=renderers,
        tooltips=[
            (stacker_name, "$name"),  # stacker name
            (x_col, f"@{x_col}"),
            (value_name, "@$name{0.0%}"),  # format as percent
        ],
    )
    p.add_tools(hover)

    # Fine-tuning the legend
    p.legend.location = "top_left"
    # p.legend.orientation = "horizontal"

    # Fine-tuning the Y axis
    p.yaxis.formatter = NumeralTickFormatter(format="0%")

    output_file(f".charts/figure_01_{lang}.html")
    show(p)

In [7]:
# Creating the Danish-language chart
figure_1(chart_data, "dk")

In [8]:
# Creating the English-language chart
figure_1(chart_data_en, "en")

For comparison purposes, I also make a `plotly` chart so that I can compare the layout:

In [9]:
import plotly.express as px
import pandas as pd


def figure_1_plotly(data: pd.DataFrame, lang: str):

    # Language-dependent labels
    if lang == "dk":
        x_col = "År"
        y_axis_label = "Andel af stemmer (%)"
        stacker_name = "Stemme"
        value_name = "Andel af stemmer"
    else:
        x_col = "Year"
        y_axis_label = "Share of votes (%)"
        stacker_name = "Vote"
        value_name = "Share of votes"

    # Identify the value columns (all except Year/År)
    stackers = [c for c in data.columns if c != x_col]
    n_unique = len(stackers)

    # Melt the dataframe → long format
    df_long = data.melt(
        id_vars=x_col,
        value_vars=stackers,
        var_name=stacker_name,
        value_name=value_name,
    )

    # Build stacked area chart
    fig = px.area(
        df_long,
        x=x_col,
        y=value_name,
        color=stacker_name,
        color_discrete_sequence=my_custom_palette[:n_unique],
        labels={
            x_col: x_col,
            value_name: y_axis_label,
            stacker_name: stacker_name,
        },
    )

    # Format y-axis as percent
    fig.update_yaxes(tickformat=".0%")

    # Legend placement similar to Bokeh's top-left
    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0,
        ),
        height=400,
    )

    # Save to HTML like Bokeh
    fig.write_html(f".charts/figure_01_{lang}_plotly.html")

    return fig

In [10]:
# Creating the Danish-language chart
figure_1_plotly(chart_data, "dk")

## Figure 2

INSERT CHART WITH TOTAL % OF VOTES FOR/AGAINST/ETC BY PARTY

In [ ]:
# Aggregating data for chart
id_cols = ["PartiGruppe", "Stemme"]
cols_keep = id_cols + ["AndelStemmer"]
chart_data = ft_afsteminger[ft_afsteminger["Stemme"] != "Fraværende"].copy()
chart_data["AntalStemmer"] = chart_data.groupby(id_cols)["Sæson"].transform("count")
chart_data["AlleStemmer"] = chart_data.groupby("PartiGruppe")["Sæson"].transform(
    "count"
)
chart_data["AndelStemmer"] = chart_data["AntalStemmer"] / chart_data["AlleStemmer"]
chart_data = chart_data.drop_duplicates(id_cols)

# Pivoting the data
chart_data = chart_data.pivot_table(
    index="PartiGruppe", columns="Stemme", values="AndelStemmer", aggfunc="sum"
).reset_index()
chart_data = chart_data.fillna(0)

# Reordering the columns
cols_order = ["PartiGruppe", "For", "Imod", "Hverken for eller imod"]
col_names = ["Parti", "For", "Imod", "Hverken for eller imod"]
chart_data = chart_data[cols_order]
chart_data.columns = col_names

# Specifying col names in English
col_names = ["Party", "For", "Against", "Neither for nor against"]
chart_data_en = chart_data.copy()
chart_data_en.columns = col_names

In [ ]:
chart_data

,Parti,For,Imod,Hverken for eller imod
0,Alternativet (ALT),1.000000,0.000000,0.000000
1,Borgernes Parti – Lars Boje Mathiesen (BP),0.000000,1.000000,0.000000
2,Danmarksdemokraterne - Inger Støjberg (DD),0.163636,0.000000,0.836364
3,Dansk Folkeparti (DF),0.287594,0.710526,0.001880
4,Det Konservative Folkeparti (KF),0.821293,0.000000,0.178707
5,Det Radikale Venstre (RV),1.000000,0.000000,0.000000
6,Enhedslisten (EL),1.000000,0.000000,0.000000
7,"Frie Grønne, Danmarks Nye Venstrefløjsparti (FG)",1.000000,0.000000,0.000000
8,Inuit Ataqatigiit (IA),0.000000,0.000000,1.000000
9,Kristendemokraterne (KD),1.000000,0.000000,0.000000


### Code below this line is WIP

Check more in the official documentation: https://docs.bokeh.org/en/latest/docs/user_guide/basic/bars.html#ug-basic-bars-stacked

In [ ]:
def figure_2(data: pd.DataFrame, lang: str):

    # Language-dependent columns & labels
    if lang == "dk":
        x_col = "Parti"
        y_axis_label = "Andel af stemmer (%)"
        stacker_name = "År"
        value_name = "Andel af stemmer"
    else:
        x_col = "Party"
        y_axis_label = "Share of votes (%)"
        stacker_name = "Year"
        value_name = "Share of votes"

    # Build ColumnDataSource
    source = ColumnDataSource(data)

    # Stackers = the year columns
    stackers = [c for c in data.columns if c != x_col]
    n_unique = len(stackers)

    p = figure(
        width=700,
        height=400,
        sizing_mode="stretch_width",
        x_axis_label=x_col,
        y_axis_label=y_axis_label,
        toolbar_location="right",
    )

    # Create stacked bars
    renderers = p.vbar_stack(
        stackers=stackers,
        x=x_col,
        width=0.8,
        color=my_custom_palette[:n_unique],
        legend_label=[str(s) for s in stackers],
        source=source,
        alpha=0.8,
    )

    # Hover tool
    hover = HoverTool(
        renderers=renderers,
        tooltips=[
            (stacker_name, "$name"),
            (x_col, f"@{x_col}"),
            (value_name, "@$name{0.0%}"),
        ],
    )
    p.add_tools(hover)

    # Format y axis as %
    p.yaxis.formatter = NumeralTickFormatter(format="0%")

    # Legend
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"

    output_file(f"figure_02_{lang}.html")
    show(p)

In [ ]:
figure_2(chart_data, "dk")

In [ ]:
fruits = ["Apples", "Pears", "Nectarines", "Plums", "Grapes", "Strawberries"]
years = ["2015", "2016", "2017"]

data = {
    "fruits": fruits,
    "2015": [2, 1, 4, 3, 2, 4],
    "2016": [5, 3, 4, 2, 4, 6],
    "2017": [3, 2, 4, 4, 5, 3],
}

data

{'fruits': ['Apples',
  'Pears',
  'Nectarines',
  'Plums',
  'Grapes',
  'Strawberries'],
 '2015': [2, 1, 4, 3, 2, 4],
 '2016': [5, 3, 4, 2, 4, 6],
 '2017': [3, 2, 4, 4, 5, 3]}

In [ ]:
ColumnDataSource(chart_data)

ColumnDataSource(id='p2340', ...)

In [ ]:
from bokeh.palettes import HighContrast3


tmp_x = chart_data["Parti"].unique().tolist()
n_x = len(tmp_x)

# p = figure(
#     x_range=chart_data["Parti"].unique(),
#     height=400,
#     toolbar_location=None,
#     tools="hover",
#     tooltips="$name @Votes: @$name",
# )

p.hbar_stack(
    stackers=tmp_x,
    y="Parti",
    height=0.9,
    color=HighContrast3[:n_x],
    source=ColumnDataSource(chart_data),
    # legend_label=[f"{year} exports" for year in years],
)

p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

ValueError: Keyword argument sequences for broadcasting must be the same length as stackers

In [ ]:
print("DONE.")

DONE.
